# Welcome to the ProPy Direct Model Example!

In this notebook, we will explore the concept of Direct Models. Direct Models estimate the time of event directly from the system state, rather than through state transitions. This approach is particularly useful for data-driven models that map sensor data to the time of an event and for physics-based models where differential equations of state transitions can be solved.

The Direct Model approach offers a faster and more efficient way to estimate the time of an event, especially for events that occur later in the simulation.

We will illustrate this concept by extending the `ThrownObject` state transition model to create a `DirectThrownObject` model. The `DirectThrownObject` model directly estimates the time when a thrown object will hit the ground from its initial state.

### Importing Modules

In [ ]:
import time
import numpy as np
from progpy.models import ThrownObject

First, let's see how estimating the time of an event works for a timeseries model. We initialize a `ThrownObject` model and calculate the time it takes to estimate the time of the event.


In [ ]:
m = ThrownObject()
x = m.initialize()
print(m.__class__.__name__, "(Direct Model)" if m.is_direct else "(Timeseries Model)")
tic = time.perf_counter()
print('Time of event: ', m.time_of_event(x, dt = 0.05))
toc = time.perf_counter()
print(f'execution: {(toc-tic)*1000:0.4f} milliseconds')

Let's take a look at how a DirectModel estimates time of event. In this case, we extend the `ThrownObject` model to include the `time_of_event` method, which is defined in Direct Models. This method allows us to directly solve the differential equations to estimate the time at which the events (falling and impact) occur. 

In [ ]:
class DirectThrownObject(ThrownObject):
    def time_of_event(self, x, *args, **kwargs):
        # calculate time when object hits ground given x['x'] and x['v']
        # 0 = x0 + v0*t - 0.5*g*t^2
        g = self.parameters['g']
        t_impact = -(x['v'] + np.sqrt(x['v']*x['v'] - 2*g*x['x']))/g

        # 0 = v0 - g*t
        t_falling = -x['v']/g
            
        return {'falling': t_falling, 'impact': t_impact}

Note that adding *args and **kwargs is optional. Having these arguments makes the function interchangeable with other models which might have arguments or keyword arguments

Now, we can estimate the time of the event for a `ThrownObject` using our `DirectThrownObject` model. Instead of simulating to a threshold, we can estimate the time of the event directly from the state. This process is significantly faster for the direct model, especially when the event occurs later in the simulation. 

In [ ]:
m = DirectThrownObject()
x = m.initialize()  # Using Initial state
# Now instead of simulating to threshold, we can estimate it directly from the state, like so
print('\n', m.__class__.__name__, "(Direct Model)" if m.is_direct else "(Timeseries Model)")
tic = time.perf_counter()
print('Time of event: ', m.time_of_event(x))
toc = time.perf_counter()
print(f'execution: {(toc-tic)*1000:0.4f} milliseconds')

Notice that execution is __MUCH__ faster for the direct model. This is even more pronounced for events that occur later in the simulation.

In this case, the `DirectThrownObject` has a defined next_state and output equation, allowing it to be used with a state estimator (e..g, Particle Filter)

## Conclusion

The Direct Model approach is a powerful tool for estimating the time of an event directly from the system state. By avoiding the process of state transitions, Direct Models can provide faster and more efficient event time estimates, particularly for events that occur later in the simulation.

Furthermore, the Direct Model approach is not limited to physics-based models. It can also be applied to data-driven models that map sensor data directly to the time of an event.

In conclusion, Direct Models offer an efficient and versatile approach for prognostics modeling, enabling faster and more direct estimations of event times.

For more ProgPy information, please refer to our ProgPy [Documentation](https://nasa.github.io/progpy/).